In [0]:
"""
以下は、100個のパイプを生成・描画・保存できるように修正した全コードです。可視化はケース数に合わせて自動で「ほぼ正方」に並べるようにし、保存先ディレクトリ（data/point_cloud）を自動作成します。
以下は、「直線ー曲線ー直線」（曲線1箇所）のパイプを生成するように修正した全コードです。
外向き（後戻りしない）符号調整や可視化、等スケール処理、100ケースの描画・保存などはそのまま引き継いでいます。
"""
import os
import numpy as np
import matplotlib.pyplot as plt
# ====== ここから：元の「きれいな円断面」コード（関数群） ======
EPS = 1e-9
def normalize(v):
    n = np.linalg.norm(v)
    if n < EPS:
        raise ValueError("ゼロに近いベクトルは正規化できません。")
    return v / n
def orthonormal_basis_from_normal(n):
    """法線 n に直交する円周基底 u,w を返す（数値的に安定）"""
    n = normalize(n)
    if abs(n[0]) < 0.9:
        a = np.array([1.0, 0.0, 0.0])
    else:
        a = np.array([0.0, 1.0, 0.0])
    u = normalize(np.cross(n, a))
    w = np.cross(n, u)
    return u, w
def sample_circle_points(center, normal, radius, num_points):
    """center・normal（断面法線）・半径から円周を整数点数でサンプリング"""
    u, w = orthonormal_basis_from_normal(normal)
    angles = np.linspace(0.0, 2.0*np.pi, num_points, endpoint=False)
    pts = center + radius * (np.cos(angles)[:,None]*u + np.sin(angles)[:,None]*w)
    return pts  # (num_points, 3)
def build_arc_geometry(start_point, start_tangent, bend_axis, bend_radius):
    """
    円弧の幾何（中心C, 基底u,v, 軸a）を構成。
    要件: bend_axis ⟂ start_tangent（直交）
    start_point = C + R*u, 接線(φ=0) = v = start_tangent
    """
    a = normalize(bend_axis)
    t = normalize(start_tangent)
    if abs(np.dot(a, t)) > 1e-6:
        raise ValueError("曲げ軸は開始接線と直交である必要があります。bend_axis ⋅ start_tangent ≠ 0")
    v = t
    u = normalize(np.cross(v, a))  # a×u = v の向き
    C = start_point - bend_radius * u
    return C, u, v, a
def map_line(start, direction):
    """直線の距離パラメータ s ↦ (中心, 接線)"""
    d = normalize(direction)
    def f(s):
        return start + d*s, d
    return f
def map_arc(start_point, start_tangent, bend_axis, bend_radius, bend_angle):
    """円弧の距離パラメータ s ↦ (中心, 接線)（s は弧長、bend_angle の符号に対応）"""
    C, u, v, a = build_arc_geometry(start_point, start_tangent, bend_axis, bend_radius)
    sign = 1.0 if bend_angle >= 0.0 else -1.0
    def f(s):
        phi = sign * (s / bend_radius)
        center = C + bend_radius * (np.cos(phi)*u + np.sin(phi)*v)
        tangent = -np.sin(phi)*u + np.cos(phi)*v
        return center, normalize(tangent)
    # 終端の幾何（更新用）
    phi_end = bend_angle
    end_point = C + bend_radius * (np.cos(phi_end)*u + np.sin(phi_end)*v)
    end_tangent = normalize(-np.sin(phi_end)*u + np.cos(phi_end)*v)
    Larc = bend_radius * abs(bend_angle)
    return f, Larc, end_point, end_tangent
def sample_segment_by_spacing(Lseg, map_func, ds, pipe_radius, num_circ_points, include_start, next_offset):
    """
    セグメント長 Lseg に対し、全体で一定間隔 ds を保つよう距離パラメータを打つ。
    - include_start: True のとき、セグメント開始位置からサンプル可能（最初のセグメント専用）
    - next_offset: セグメント開始から次サンプルまでの残距離（前セグメントからのキャリー）
    戻り値: (points, next_offset_new, last_center, last_tangent)
    """
    pts_list = []
    # セグメント開始からの最初のサンプル位置
    if include_start:
        s = next_offset  # 初回は 0 から可能
    else:
        s = next_offset if next_offset > EPS else ds  # 境界重複回避
    last_center, last_tangent = None, None
    while s < Lseg - EPS:
        center, tangent = map_func(s)
        circle_pts = sample_circle_points(center, tangent, pipe_radius, num_circ_points)
        pts_list.append(circle_pts)
        last_center, last_tangent = center, tangent
        s += ds
    # 次セグメントの next_offset を更新（このセグメント終端から次サンプルまでの距離）
    next_offset_new = s - Lseg  # in [0, ds)
    if len(pts_list) == 0:
        return np.empty((0,3)), next_offset_new, last_center, last_tangent
    return np.vstack(pts_list), next_offset_new, last_center, last_tangent
# ====== ここまで：元コード（関数群） ======
def compute_global_equal_maxspan_limits(pointclouds, pad_ratio=0.05):
    """
    複数点群をまとめた境界から、中心と最大幅を求め、
    各軸同じ幅（= 最大幅 × (1+pad_ratio)）の立方体範囲を返す。
    """
    all_pts = np.vstack(pointclouds)
    mins = all_pts.min(axis=0)  # [xmin, ymin, zmin]
    maxs = all_pts.max(axis=0)  # [xmax, ymax, zmax]
    center = 0.5 * (mins + maxs)
    max_span = float(np.max(maxs - mins))  # 3軸中の最大幅
    half = 0.5 * max_span * (1.0 + pad_ratio)  # 余白込みの半幅
    xlim = (center[0] - half, center[0] + half)
    ylim = (center[1] - half, center[1] + half)
    zlim = (center[2] - half, center[2] + half)
    return xlim, ylim, zlim
# ====== 修正版：外向き接続を保証する generate 関数（直線-円弧-直線） ======
def generate_pipe_from_params(
    params,                 # shape (3,2): [L,0], [θ,R], [L,0]
    bend_axis,              # shape (3,) or (1,3): 曲げ軸ベクトル
    pipe_radius=0.05,
    target_spacing=0.02,
    start_pos=np.array([0.0, 0.0, 0.0]),
    start_dir=np.array([1.0, 0.0, 0.0]),
    enforce_outward=True,           # 外向きに強制
    clip_angle_to_pi=True           # |θ| を π 以内にクリップ
):
    """
    直線–円弧–直線 を、(3,2) パラメータで生成。
    - enforce_outward=True のとき、各円弧角度 θ は sin(θ) >= 0 になるよう符号を自動調整
      （弧の変位 Δ の接線方向成分 R sinθ を非負にし、開始端から“前向き＝外向き”に進める）
    - clip_angle_to_pi=True のとき、|θ| > π を π にクリップ（極端なループ防止）
    """
    params = np.asarray(params, dtype=float)
    if params.shape != (3,2):
        raise ValueError("params は形状 (3,2) にしてください。")
    bend_axis = np.asarray(bend_axis, dtype=float)
    if bend_axis.shape == (1,3):
        bend_axis = bend_axis[0]
    if bend_axis.shape != (3,):
        raise ValueError("bend_axis は形状 (3,) もしくは (1,3) にしてください。")
    # 円周方向の点数を決定（円周間隔 ≈ target_spacing になるよう整数分割）
    num_circ_points = max(8, int(round(2.0*np.pi * pipe_radius / target_spacing)))
    ds = (2.0*np.pi * pipe_radius) / num_circ_points  # 実際の間隔
    pts_all = []
    current_pos = np.array(start_pos, dtype=float)
    current_dir = normalize(np.array(start_dir, dtype=float))
    next_offset = 0.0
    # セグメントの順序とタイプ（直線-円弧-直線）
    segment_types = ["line", "arc", "line"]
    for i, seg_type in enumerate(segment_types):
        p0, p1 = params[i, 0], params[i, 1]
        if seg_type == "line":
            L = float(p0)
            if L < -EPS:
                raise ValueError(f"直線長が負です: L={L}")
            f_line = map_line(current_pos, current_dir)
            include_start = (i == 0)
            pts, next_offset, _, _ = sample_segment_by_spacing(
                Lseg=L, map_func=f_line, ds=ds, pipe_radius=pipe_radius,
                num_circ_points=num_circ_points, include_start=include_start, next_offset=next_offset)
            if pts.size > 0:
                pts_all.append(pts)
            current_pos = current_pos + current_dir * L
        elif seg_type == "arc":
            theta = float(p0)
            R = float(p1)
            if R <= EPS:
                raise ValueError(f"円弧半径が非正です: R={R}")
            # “外向き”を保証するために角度符号を自動調整
            if enforce_outward and np.sin(theta) < 0:
                theta = -theta
            if clip_angle_to_pi and abs(theta) > np.pi:
                theta = np.sign(theta) * (np.pi - 1e-6)
            f_arc, Larc, end_point, end_tangent = map_arc(
                start_point=current_pos,
                start_tangent=current_dir,
                bend_axis=bend_axis,
                bend_radius=R,
                bend_angle=theta
            )
            pts, next_offset, _, _ = sample_segment_by_spacing(
                Lseg=Larc, map_func=f_arc, ds=ds, pipe_radius=pipe_radius,
                num_circ_points=num_circ_points, include_start=False, next_offset=next_offset)
            if pts.size > 0:
                pts_all.append(pts)
            current_pos = end_point
            current_dir = end_tangent
        else:
            raise RuntimeError("未知のセグメントタイプ")
    if len(pts_all) == 0:
        return np.empty((0,3))
    return np.vstack(pts_all)
# ====== ここから：100個を作成・可視化 ======
if __name__ == "__main__":
    np.random.seed(42)  # 再現性のため
    pipe_radius = 0.05
    target_spacing = 0.02  # 等間隔目標（リング間隔）
    # 曲げ軸は Z軸（XY平面曲げ）。直交条件を常に満たす安全な設定。
    bend_axis_template = np.array([0.0, 0.0, 1.0])
    def make_random_params():
        # 直線長: 0.5〜1.5, 0.5〜1.5
        L1 = np.random.uniform(0.5, 1.5)
        L2 = np.random.uniform(0.5, 1.5)
        # 円弧角: 30°〜180°（符号はランダムだが、generate内で外向きに自動調整）
        ang = np.deg2rad(np.random.uniform(30, 180)) * np.random.choice([1, -1])
        # 半径: 0.2〜0.5
        R = np.random.uniform(0.2, 0.5)
        return np.array([
            [L1, 0.0],
            [ang, R],
            [L2, 0.0]
        ], dtype=float)
    # 100ケース分生成
    n_cases = 100
    params_list = [make_random_params() for _ in range(n_cases)]
    # 各ケースの点群生成
    pointclouds = []
    for i, params in enumerate(params_list):
        points = generate_pipe_from_params(
            params=params,
            bend_axis=bend_axis_template,
            pipe_radius=pipe_radius,
            target_spacing=target_spacing,
            start_pos=np.array([0.0, 0.0, 0.0]),
            start_dir=np.array([1.0, 0.0, 0.0]),
            enforce_outward=True,        # 外向きを強制
            clip_angle_to_pi=True        # 角度クリップ
        )
        pointclouds.append(points)
        print(f"[{i}] params(before adjust)=\n{params}\n -> points: {points.shape}")
    # 表示範囲を全ケースで統一（各軸幅＝最大幅に揃える）
    xlim, ylim, zlim = compute_global_equal_maxspan_limits(pointclouds, pad_ratio=0.05)
    # 可視化（ケース数に応じてほぼ正方に並べる）
    rows = int(np.ceil(np.sqrt(n_cases)))
    cols = int(np.ceil(n_cases / rows))
    fig = plt.figure(figsize=(cols * 2.2, rows * 2.2))
    for i, pts in enumerate(pointclouds):
        ax = fig.add_subplot(rows, cols, i+1, projection='3d')
        ax.scatter(pts[:,0], pts[:,1], pts[:,2], s=0.6, c='navy', alpha=1.0, linewidths=0)
        ax.set_title(f"Case {i}", fontsize=8)
        ax.set_xlim(*xlim); ax.set_ylim(*ylim); ax.set_zlim(*zlim)
        try:
            ax.set_box_aspect([1,1,1])  # 等方比
        except Exception:
            pass
        ax.set_xticks([]); ax.set_yticks([]); ax.set_zticks([])
        ax.set_facecolor('white')
    plt.tight_layout()
    plt.show()
    # ファイル保存（ディレクトリ作成込み）
    out_dir = "data/point_cloud"
    os.makedirs(out_dir, exist_ok=True)
    for i, pts in enumerate(pointclouds):
        np.savetxt(os.path.join(out_dir, f"pipe_case_{i:02d}.xyz"),
                   pts, delimiter=' ', header='X Y Z', comments='')
"""
ポイント
- n_cases を 100 に変更し、描画用のグリッド（rows, cols）をケース数に合わせて自動調整しています（100なら 10×10）。
- 保存先ディレクトリ data/point_cloud を自動作成します（存在しない場合のエラー回避）。
- 可視化の各サブプロットを小さめにし、全体の図サイズを rows・cols に応じて拡大することで 100ケースでも表示できるようにしています。必要に応じて figsize や散布点サイズ s を調整してください。
"""